In [22]:
import numpy as np
from scipy.optimize import minimize

這裡假設是

工段序號: 1 2 3 4 (I)

每個工段耗時: 5 4 3 3 小時

人員序號: 1 2 3 4 5 (J)

總人數: 10人

每個人的工作時數限制: 8 (hrs)

In [23]:
I = [1, 2, 3, 4]  # 工段序號
J = np.arange(1,11)  # 人員序號
x = [18, 15, 10, 10]  # 每個工段的耗時(hrs)
N = 10  # 總人數
ML = [8 for _ in range(len(J))]  # 每個人的工作時數限制(hrs)
S = [ 1, 2, 3, 4, None ,1 ,3 ,4 ,3 ,2]  # 每個人的技能對應工段


In [24]:
def objective(n_i,sgn = -1): #目標函數
    n_i = np.maximum(n_i, 1)
    
    return sgn*np.min([3600/(x[i]/n_i[i])*0.85 for i in range(len(I))])

In [25]:
def constraint1(n_i):     #限制1:保證所有人總和是N
    return np.sum(n_i)-N

def constraint2(n_i):    #限制2:每個人的工作時間不超過他們的ML
    total_time = [0]*len(J)
    for j in range(len(J)):
        for i in range(len(I)):
            if S[j] == i:
                total_time[j] += n_i[i]
                
    return [ML[j]-total_time[j] for j in range(len(J))]
    # 計算剩餘工作時數
#     remaining_time = [ML[j] - total_time[j] for j in range(len(J))]
    
#      #輸出每個人的剩餘工作時數
#     for j in range(len(J)):
        
#         print(f"Person {j}: ML = {ML[j]}, Total time = {total_time[j]}, Remaining time = {remaining_time[j]}")
    
#     return remaining_time

def constraint3(n_i):
    n_i = np.maximum(n_i, 1)
    constraints = []
    for a in range(len(I)):
        for b in range(len(I)):
            if a < b :
                constraints.append((x[a]/n_i[a]) - (x[b]/n_i[b]))
    return constraints

In [26]:
#起始值

n_i0 = [N/len(I)]*len(I)

con1 = {'type':'eq','fun':constraint1}
con2 = {'type':'ineq','fun':lambda n_i: np.array(constraint2(n_i))}
con3 = {'type':'ineq','fun':lambda n_i: np.array(constraint3(n_i))}
cons = [con1, con2, con3]

In [27]:
#邊界
bounds = [(1,N) for _ in I]

In [28]:
#最大化objective

solution = minimize(objective, n_i0 , method = 'SLSQP', bounds = bounds, constraints = cons)

n_i_opt = solution.x
n_i_opt = [round(values,2) for values in n_i_opt]#取到小數點第2位


print("最佳人數配置: ",n_i_opt)
print("最大化瓶頸工段產量: ",round(-solution.fun,2))


最佳人數配置:  [3.4, 2.83, 1.89, 1.89]
最大化瓶頸工段產量:  577.36
